In [ ]:
! jt -t chesterish # chesterish 테마로 변경합니다.

In [ ]:
import pandas as pd
import os

def load(
    filter_size: '60, 90, 180', 
    target_size: '1, 7, 30, 180', 
    stride: '1, 2, 3',
    drop_suffix: '각 변수 끝에 붙은 번호를 제거할지 여부'=True,
    path='/Users/Namin/Documents/CAP/CapstoneUOS/Data/data_variants',) -> (pd.DataFrame, pd.DataFrame):
    
    print(f'Setting: filter_size({filter_size})\ttarget_size({target_size})\tstride({stride})\tdrop_suffix({drop_suffix})')
    X_name = f'fs({filter_size})_ts({target_size})_st({stride}).csv'
    y_name = f'fs({filter_size})_ts({target_size})_st({stride})_label.csv'
    
    print('Load feature data...', end='\t')
    X = pd.read_csv(os.path.join(path, X_name))
    print('loaded!')
    print('Load label data...', end='\t')
    y = pd.read_csv(os.path.join(path, y_name))
    print('loaded!')
    
    if drop_suffix:
        X.columns = list(map(lambda x: x.split('.')[0], X.columns.tolist()))
        y.columns = list(map(lambda x: x.split('.')[0], y.columns.tolist()))

    return X, y

In [ ]:
filter_size = 180
target_size = 90
stride = 1
X, y = load(filter_size, target_size, stride)

In [ ]:
features = ['n_hashtage', 'video_num', 'no_upload_interval',
            'duration', 'video_n_view', 'n_dislike', 'n_like', 'n_comment',
            'view_diff',
            'sub_diff']
target_features = ['sub_diff']

X = X[features]
y = y[target_features]
y = pd.DataFrame(y.iloc[:,-1], columns=['sub_diff'])

In [ ]:
# Split

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

random_state = 42
test_size = 0.2

X_trn, X_test, y_trn, y_test = train_test_split(
    X, y, 
    shuffle=True, 
    test_size=test_size, 
    random_state=random_state
) 

X_train, X_valid, y_train, y_valid = train_test_split(
    X_trn, y_trn, 
    shuffle=True, 
    test_size=test_size, 
    random_state=random_state
) 

### 스케일링 필요 시 다음을 진행(타깃에 대한 스케일링은 진행되지 않음)
scaler = MinMaxScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
X_valid = pd.DataFrame(scaler.transform(X_valid), columns=X_valid.columns)
X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

In [ ]:
print("X_train's shape: ", X_train.shape) 
print("y_train's shape: ", y_train.shape) 
print("X_valid's shape: ", X_valid.shape)
print("y_valid's shape: ", y_valid.shape)
print("X_test's shape: ", X_test.shape)
print("y_test's shape: ", y_test.shape)

In [ ]:
# 데이터 구성 살피기
import matplotlib.pyplot as plt
plt.style.use('ggplot')

h = 3 # graph's height(length)

for n in range(10): # 1부터 x_train.shape[0]까지
    fig = plt.figure(figsize=(h*len(set(X_train.columns.tolist())), h))
    for i, train_var in enumerate(set(X_train.columns.tolist())):
        plt.subplot(1, len(set(X_train.columns.tolist())), i+1)
        plt.plot(X_train[train_var].iloc[n].values)
        plt.title(str(train_var))

    fig = plt.figure(figsize=(h*len(set(y_train.columns.tolist())), h))
    for i, label_var in enumerate(set(y_train.columns.tolist())):
        plt.subplot(1, len(set(y_train.columns.tolist())), i+1)
        plt.plot(y_train[label_var].iloc[n], marker='o')
        plt.title("GOAL: " + str(label_var))
        
    plt.show()

In [ ]:
n1 = 10 # number of train features
n2 = 1 # number of target features

x_train_copy = X_train.fillna(0).values.reshape(X_train.shape[0], n1, -1).swapaxes(1, 2)
y_train_copy = y_train.fillna(0).values.reshape(y_train.shape[0], n2, -1).swapaxes(1, 2)
x_val_copy = X_valid.fillna(0).values.reshape(X_valid.shape[0], n1, -1).swapaxes(1, 2)
y_val_copy = y_valid.fillna(0).values.reshape(y_valid.shape[0], n2, -1).swapaxes(1, 2)
x_test_copy = X_test.fillna(0).values.reshape(X_test.shape[0], n1, -1).swapaxes(1, 2)
y_test_copy = y_test.fillna(0).values.reshape(y_test.shape[0], n2, -1).swapaxes(1, 2)

In [ ]:
print("x_train_copy's shape: ", x_train_copy.shape) 
print("y_train_copy's shape: ", y_train_copy.shape) 
print("x_val_copy's shape: ", x_val_copy.shape)
print("y_val_copy's shape: ", y_val_copy.shape)
print("x_test_copy's shape: ", x_test_copy.shape)
print("y_test_copy's shape: ", y_test_copy.shape)

In [ ]:
# 모델 만들기
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, TimeDistributed, Conv1D, MaxPooling1D, Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam, RMSprop

filters = len(features)
output_num = len(target_features)

model = Sequential()
model.add(TimeDistributed(Dense(8, activation='softmax'),
          input_shape=(x_train_copy.shape[-2:])))
model.add(Conv1D(filters=16, kernel_size=3, strides=1, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(128))
model.add(Dense(64))
model.add(Dense(16))
model.add(Dense(output_num))

def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

model.compile(loss=['mae'],
              metrics=[tf.keras.metrics.RootMeanSquaredError()],
              optimizer=Adam(lr=0.0001))
model.summary()

In [ ]:
# 기록하기
import time

filename='cnn_{}_{}_{}_2features.h5'.format(filter_size, target_size, stride)

callback_list = [tf.keras.callbacks.ModelCheckpoint(filepath='Checkpoint/{}'.format(filename),
                                                    monitor='val_loss',
                                                    verbose=1,
                                                    save_best_only=True,
                                                    mode='min'),
                 #tf.keras.callbacks.TensorBoard(log_dir='TensorBoard/cnn/{}'.format(time.asctime())),
                 tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                  patience=5),
                 #tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                 #                                     factor=0.1,
                 #                                     min_lr=1e-4, 
                 #                                     patience=0)
                ]

EPOCHS = 100
BATCH_SIZE = 32 

history2 = model.fit(x_train_copy, y_train_copy,
                     epochs=EPOCHS,
                     batch_size=BATCH_SIZE,
                     validation_data=(x_val_copy, y_val_copy),
                     callbacks=callback_list)

In [ ]:
UPDATED_EPOCHS = 7 + 1

In [ ]:
## 모델 성능 그래프로 나타내기

import numpy as np

epochs = np.arange(1, UPDATED_EPOCHS)
fig, axes = plt.subplots(figsize=(20, 5))

plt.subplot(1, 2, 1)
plt.plot(epochs, history2.history['loss'], label='train_loss')
plt.plot(epochs, history2.history['val_loss'], label='val_loss')
plt.xlabel('epochs', fontdict={'size': 10})
plt.ylabel('loss', fontdict={'size': 10})
plt.title("MAE")
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epochs, history2.history['root_mean_squared_error'], label='train_loss')
plt.plot(epochs, history2.history['val_root_mean_squared_error'], label='val_loss')
plt.xlabel('epochs', fontdict={'size': 10})
plt.ylabel('loss', fontdict={'size': 10})
plt.title("RMSE")
plt.legend()

plt.show()

In [ ]:
# 모델 저장하고 복원하기
from tensorflow.keras.models import load_model

filename = 'cnn_{}_{}_{}_2features.h5'.format(filter_size, target_size, stride)

model.save('Model/{}'.format(filename))
res_model = load_model('Model/{}'.format(filename))
res_model.load_weights('Model/{}'.format(filename))
#res_model.evaluate(x_test_copy, y_test_copy)
mae, rmse = model.evaluate(x_test_copy, y_test_copy, verbose=2)
print("MAE of Untrained Model: {:5.2f}%".format(mae))
print("RMSE of Untrained Model: {:5.2f}%".format(rmse))

In [ ]:
# 테스트 세트로 예측하기
test_predict = model.predict(x_test_copy)

In [ ]:
# 예측한 것 시각화하기 (한 개 다 합친 것)

actual = pd.DataFrame(y_test_copy[:,0,0], columns=['actual'])
pred = pd.DataFrame(test_predict[:,0], columns=['pred'])
result = pd.concat([actual, pred], axis=1)
ord_result = result.sort_values(by=['actual']) # 순차적으로
start, end = 7500, 8000
my_result = ord_result.iloc[start:end] # 보고 싶은 범위만

fig, axes = plt.subplots(figsize=(20, 15))

plt.subplot(3, 1, 1)
total = [j for j in range(y_test_copy.shape[0])]
plt.plot(total, result['actual'], 'r', label='actual')
plt.plot(total, result['pred'], 'b', label='predicted', alpha=0.7)
plt.title("All of the Test Datas")
plt.legend()

plt.subplot(3, 1, 2)
total = [j for j in range(ord_result.shape[0])]
plt.plot(total, ord_result['actual'], 'r', label='actual')
plt.plot(total, ord_result['pred'], 'b', label='predicted', alpha=0.7)
plt.title("All of the Test Datas (ordered)")
plt.legend()

plt.subplot(3, 1, 3)
total = [j for j in range(my_result.shape[0])]
plt.plot(total, my_result['actual'], 'r', label='actual')
plt.plot(total, my_result['pred'], 'b', label='predicted', alpha=0.7)
plt.title("Some Test Datas (ordered)")
plt.legend()

plt.show()

In [ ]:
good_a, good_p, bad_a, bad_p = [], [], [], []
mae = int(mae)

for i in range(result.shape[0]):
    a = result['actual'].iloc[i]
    p = result['pred'].iloc[i]

    if abs(a-p) <= mae:
        good_a.append(a)
        good_p.append(p)
    else:
        bad_a.append(a)
        bad_p.append(p)

good_a = pd.DataFrame(good_a, columns=['actual'])
good_p = pd.DataFrame(good_p, columns=['pred'])
good_result = pd.concat([good_a, good_p], axis=1)
good_ord_result = good_result.sort_values(by=['actual']) # 순차적으로

bad_a = pd.DataFrame(bad_a, columns=['actual'])
bad_p = pd.DataFrame(bad_p, columns=['pred'])
bad_result = pd.concat([bad_a, bad_p], axis=1)
bad_ord_result = bad_result.sort_values(by=['actual']) # 순차적으로

print("total test samples:", result.shape[0], "개")
per_mae = np.round((good_result.shape[0]/result.shape[0])*100, 2)
print("good mae result / total result (%):", per_mae)

fig, axes = plt.subplots(figsize=(20, 20)) # Test MAE보다 작거나 같은 것만

plt.subplot(4, 1, 1)
total = [j for j in range(good_result.shape[0])]
plt.plot(total, good_result['actual'], 'r', label='actual')
plt.plot(total, good_result['pred'], 'b', label='predicted', alpha=0.7)
plt.legend()
plt.title("Good Predictions (error less than MAE {}".format(mae)+ ")")

plt.subplot(4, 1, 2)
total = [j for j in range(good_ord_result.shape[0])]
plt.plot(total, good_ord_result['actual'], 'r', label='actual')
plt.plot(total, good_ord_result['pred'], 'b', label='predicted', alpha=0.7)
plt.legend()
plt.title("Good Predictions (error less than MAE {}".format(mae)+ ") (ordered)")

plt.subplot(4, 1, 3)
total = [j for j in range(bad_result.shape[0])]
plt.plot(total, bad_result['actual'], 'r', label='actual')
plt.plot(total, bad_result['pred'], 'b', label='predicted', alpha=0.7)
plt.legend()
plt.title("Bad Predictions (error more than MAE {}".format(mae)+ ")")

plt.subplot(4, 1, 4)
total = [j for j in range(bad_ord_result.shape[0])]
plt.plot(total, bad_ord_result['actual'], 'r', label='actual')
plt.plot(total, bad_ord_result['pred'], 'b', label='predicted', alpha=0.7)
plt.legend()
plt.title("Bad Predictions (error ore than MAE {}".format(mae)+ ") (ordered)")

plt.show()

In [ ]:
start = 40000
end = 45000

column_classes = X.columns.unique().tolist()
channel = pd.DataFrame(scaler.transform(X.loc[start:end, :]), columns=X.columns)
channel_list = channel.values.reshape(channel.shape[0], filter_size, -1) # (end-start, features, filters)

channel_real = y.loc[start:end, :].values.flatten() # (end-start, 1)
channel_pred = model.predict(channel_list).flatten() # (end-start, 1)
pred = pd.DataFrame(dict(real=channel_real, pred=channel_pred))

In [ ]:
import seaborn as sns

plt.figure(figsize=(15, 5))
to_plot = pred.melt()
to_plot['ticks'] = [i for i in range(pred.shape[0])] * 2
sns.lineplot(x='ticks', y='value', hue='variable', data=to_plot)
plt.title('Prediction for One Channel')
plt.show()